1.特徴量の作成  
  1. `game_session` 毎の集計データを作成。例として以下のような項目を集計。  
    - プレイ時間等の連続変数（平均、最大といった基本的な集計）。  
    - `title`, `world`, `type` といった `game_session` 毎に1つだけの値を持つカテゴリ変数  
      - 加えて `type` = 'Assessment' （つまり Assessment 受講）に関してはその時のスコアも集計する  
    - 'title` と `event_code' の組み合わせといった1つの `game_session` で様々な値を持つカテゴリ変数  
    
  2. Assessment を1度以上受講しているユーザ (≈`installation_id` を期待） に対して、受講より過去の 'game_session' の集計値より特徴量を作成する。  
    - 連続変数に対して移動平均値、中央値、最大値、etc...  
      - 上記に関しては更に `title` や `world` といったカテゴリ変数別の集計項目も追加。`title` = XXX に対するプレイ時間の合計とか。  
      - Assessment の受講履歴は `title` 別に正解／不正解、不正解のカテゴリ（独自に追加、"No error" "1 error" "2 errors or more" みたいに）を作成、カテゴリ別でも正解／不正解の集計    
    - 連続変数系の特徴量に関しては、それらの自然対数の特徴量も追加  
      - 0 は np.nan になり厄介なので最小値で補正。  
      
  3. 不要な特徴量を削る  
    - 1000個を超える特徴量が出来てしまったが「分散0」みたいなゴミは除外。  
    - 相関係数が高いペアが居る連続変数は片方を削除。それでも750くらい残った。  
    - この時点で残り時間が少ない！適当に RandomForestClassifer を使い、`feature_importance` が一定値以上の特徴量を全部突っ込む、という荒っぽいやり方で妥協。  

2. モデリング  
  1. 使用するアルゴリズム  
    - 最終的には LightGBM.   
    - SVC, K-Neighbor も少し試したが、時間の制約で深く試せない。  
  2. 工夫：過学習だけは防ぎたい！  
    - train.csv より作成した特徴量に対して 5-hold Stratified-Group K-hold.  
      - 教師ラベルで Stratified し`installation_id` で Group.  
      - ある `installation_id` が訓練データと検証データの両方に登場すると Cross Validation のスコアが過大評価されるのでは？と懸念した。  
      - Stratified-Group K-hold のコードはコピペで拝借（先人に感謝）。  
    - test.csv より複数回 Assessment を受講した `installation_id` を抽出。その人たちのデータをテストデータに使用してモデルのフィット感を試した。  

3. 個人的な反省  
  - 特徴量エンジニアリングにもっと時間を割きたかった
  - submit エラー多すぎて無駄が多い（1回数時間要するsubmitを15回、成功は6回だけ、データ分析どうこう以前の問題）
  - 純粋にコーディングの生産性が低い（やりたい事の量＞（あまりに高い壁）＞出来る事の量）
  - フルスクラッチにこだわり過ぎた


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.